# API DE DATACHILE
DataChile posee una API de datos que entrega la información para construir los gráficos y generar todos los textos del sitio.

## PRIMEROS PASOS
DataChile utiliza para su funcionamiento como capa lógica Mondrian-REST, que es un componente del lado del servidor que permite la creación de APIs HTTP para acceder a una base de datos especificando la estructura lógica de la información.

Para utilizar la API de DataChile, es necesario conocer algunos conceptos que facilitarán su uso, los cuáles explicaremos a continuación.

**Cubo**

Un cubo es una colección de dimensions y measures.

En DataChile, cada dataset está agrupado en un cubo. El diseño del lenguaje de consultas a la API está centrado en torno a 3 operaciones: Drilldown, Cut, Measure. Para ejemplificar los conceptos, se utilizará el dataset exports -que representa las exportaciones de Chile- como referencia.

**Drilldown**

Algunas definiciones previas:
Un member es un valor dentro de una dimensión determinada por un set de valores. La jerarquía género tiene los miembros 'Masculino' y 'Femenino'. 'Concepción', 'Bío-Bío' y 'Chile' son todos miembros de la jerarquía Geográfica.
Un level es un conjunto de miembros que se encuentran a la misma distancia del inicio de la jerarquía. Concepción, Tucapel, Arica, se encuentran en el mismo level.
Una hierarchy es un conjunto de miembros organizados estructuradamente para su análisis. Por ejemplo, la jerarquía Geográfica consiste de nombre de comuna, regiones y país. La jerarquización permite obtener valores intermedios: Total de exportaciones de la región de Valparaíso es la suma de todas las exportaciones de las comunas de dicha región.
Una dimension es un conjunto de jerarquías que discriminan el mismo atributo (Por ejemplo, el día que se produjo una exportación)
Un drilldown permite pasar de un nivel de detalle al siguiente. La estructura de un drilldown para las consultas en el cubo es [Dimension].[Hierarchy].[Level]

**Medición**

Variable escalar asociada con un valor particular en los datos del cubo.

**Corte**

Permite al usuario especificar un filtro, que restringe las tuplas disponibles del cubo. Por ejemplo, en exports, cortar el cubo en torno al miembro [Region].[8] en la dimensión Geography, sólo considerará celdas de la región de Bío-Bío. Múltiples cortes pueden ser especificados en una simple consulta.



In [3]:
import pandas as pd
from datachile import ChileCube

client = ChileCube()

In [17]:
life_expectancy = client.get(
    "life_expectancy", 
    {
        "drilldowns": [
            ["Date", "Date", "Year"],
            ["Geography", "Geography", "Region"],
            ['Sex', 'Sex', 'Sex']
        ],
        "measures": ["Number of records",
                     "Life Expectancy AVG",
                     "Mortality rate per 100 inhabitants AVG"
                    ],
        "parents": True
    },
    df=True
)
life_expectancy.head()

,ID Region,ID Sex,ID Year,Life Expectancy AVG,Mortality rate per 100 inhabitants,Number of records,Region,Sex,Year
0,1,0,2010,0.000000,0.00,0.0,Tarapacá,No data,2010
1,1,1,2010,75.739998,0.36,1.0,Tarapacá,Female,2010
2,1,2,2010,75.739998,0.45,1.0,Tarapacá,Male,2010
3,2,0,2010,0.000000,0.00,0.0,Antofagasta,No data,2010
4,2,1,2010,78.750000,0.44,1.0,Antofagasta,Female,2010


In [18]:
client.get_measures("life_expectancy")

[{'name': 'Number of records',
  'caption': 'Number of records',
  'annotations': {},
  'full_name': '[Measures].[Number of records]',
  'aggregator': 'COUNT'},
 {'name': 'Life Expectancy AVG',
  'caption': 'Life Expectancy AVG',
  'annotations': {'es_element_caption': 'Expectativa de vida promedio'},
  'full_name': '[Measures].[Life Expectancy AVG]',
  'aggregator': 'AVG'},
 {'name': 'Mortality rate per 100 inhabitants AVG',
  'caption': 'Mortality rate per 100 inhabitants',
  'annotations': {'es_element_caption': 'Tasa de mortalidad cada 100 habitantes'},
  'full_name': '[Measures].[Mortality rate per 100 inhabitants AVG]',
  'aggregator': 'AVG'}]

In [27]:
life_expectancy = (life_expectancy.rename(columns={'Life Expectancy AVG': 'life_expectancy_avg'})
                                  .loc[lambda x: x['Sex'] != 'No data', ['Year', 'Region', 'Sex', 'life_expectancy_avg']]
                  )

In [28]:
life_expectancy.head()

,Year,Region,Sex,life_expectancy_avg
1,2010,Tarapacá,Female,75.739998
2,2010,Tarapacá,Male,75.739998
4,2010,Antofagasta,Female,78.750000
5,2010,Antofagasta,Male,73.339996
7,2010,Atacama,Female,81.190002


In [29]:
life_expectancy.describe(include='all')

,Year,Region,Sex,life_expectancy_avg
count,150,150,150,150.000000
unique,5,15,2,NaN
top,2014,Arica y Parinacota,Male,NaN
freq,30,10,75,NaN
mean,NaN,NaN,NaN,77.531666
std,NaN,NaN,NaN,3.195902
min,NaN,NaN,NaN,73.239998
25%,NaN,NaN,NaN,74.480003
50%,NaN,NaN,NaN,75.934998
75%,NaN,NaN,NaN,80.349998


In [33]:
life_expectancy.query('Region == "Antofagasta"')

,Year,Region,Sex,life_expectancy_avg
4,2010,Antofagasta,Female,78.750000
5,2010,Antofagasta,Male,73.339996
49,2011,Antofagasta,Female,78.750000
50,2011,Antofagasta,Male,73.339996
94,2012,Antofagasta,Female,78.750000
95,2012,Antofagasta,Male,73.339996
139,2013,Antofagasta,Female,78.750000
140,2013,Antofagasta,Male,73.339996
184,2014,Antofagasta,Female,78.750000
185,2014,Antofagasta,Male,73.339996


In [30]:
life_expectancy.query('Sex == "No data"')['life_expectancy_avg'].sum()

0.0

In [32]:
life_expectancy.pivot_table(index=['Region'],
                            columns='Year',
                            values='life_expectancy_avg',
                            aggfunc='mean'
                           )

Year,2010,2011,2012,2013,2014
Region,,,,,
Antofagasta,76.044998,76.044998,76.044998,76.044998,76.044998
Araucanía,77.495003,77.495003,77.495003,77.495003,77.495003
Arica y Parinacota,78.644997,78.644997,78.644997,78.644997,78.644997
Atacama,78.235001,78.235001,78.235001,78.235001,78.235001
Aysén,78.864998,78.864998,78.864998,78.864998,78.864998
Biobío,77.384998,77.384998,77.384998,77.384998,77.384998
Coquimbo,79.074997,79.074997,79.074997,79.074997,79.074997
Los Lagos,76.715000,76.715000,76.715000,76.715000,76.715000
Los Ríos,77.105000,77.105000,77.105000,77.105000,77.105000
